In [1]:
sc.stop()
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

conf = SparkConf().setAppName("zzzZ")\
                  .setMaster("spark://10.120.26.200:7077")\
                  .set("spark.executor.memory", '6g')\
                  .set('spark.cores.max', '4')

sc = SparkContext(conf=conf)
spark = SparkSession.builder.appName('sql coming~').getOrCreate()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,None,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# RDD創建表 - createDataFrame

def create_df_from_rdd():
    spark_rdd = spark.sparkContext.parallelize([
        (1, "Suny", 10, "water"),
        (2, "Miya", 30, "drink"),
        (3, "Nana", 26, "berry")])
 
    # 設置屬性
    schema = StructType([StructField("id", LongType(), True),
                         StructField("name", StringType(), True),
                         StructField("age", LongType(), True),
                         StructField("text", StringType(), True)])
    
    # 建立DataFrame
    df_from_rdd = spark.createDataFrame(spark_rdd, schema)
    
    # 建立臨時表
    df_from_rdd.createOrReplaceTempView("myTempView")
    # 使用SQL
    data = spark.sql("select * from myTempView")
    data.show()
    
create_df_from_rdd()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+---+-----+
| id|name|age| text|
+---+----+---+-----+
|  1|Suny| 10|water|
|  2|Miya| 30|drink|
|  3|Nana| 26|berry|
+---+----+---+-----+

In [16]:
# RDD創建表 - toDF

def create_df_from_rdd():
    spark_rdd = spark.sparkContext.parallelize([
        (1, "Suny", 10, "water"),
        (2, "Miya", 30, "drink"),
        (3, "Nana", 26, "berry")])
 
    # 設置屬性
    schema = StructType([StructField("id", LongType(), True),
                         StructField("name", StringType(), True),
                         StructField("age", LongType(), True),
                         StructField("text", StringType(), True)])
    
    # toDF 建立
    df_from_rdd = spark_rdd.toDF(schema)
    
    # 建立臨時表
    df_from_rdd.createOrReplaceTempView("myTempView")
    # 使用SQL
    data = spark.sql("select * from myTempView")
    return data
    
create_df_from_rdd().show()
create_df_from_rdd().printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+---+-----+
| id|name|age| text|
+---+----+---+-----+
|  1|Suny| 10|water|
|  2|Miya| 30|drink|
|  3|Nana| 26|berry|
+---+----+---+-----+

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- text: string (nullable = true)

In [15]:
# RDD創建表 - Row

def create_df_from_rdd():
    
    row =Row("id","name","age","text")
    a = [1,2,3,3]
    b = ["Miya", "Nana", "Nana"]
    c = [10,30,26]
    d = ["water", "drink", "berry"]
    spark_rdd = spark.sparkContext.parallelize([row(a[i],b[i],c[i],d[i]) for i in range(3)])
 
    # 設置屬性
    schema = StructType([StructField("id", StringType(), True),
                         StructField("name", StringType(), True),
                         StructField("age", StringType(), True),
                         StructField("text", StringType(), True)])
    
    # toDF 建立
    df_from_rdd = spark_rdd.toDF(schema)
    
    # 建立臨時表
    df_from_rdd.createOrReplaceTempView("myTempView")
    # 使用SQL
    data = spark.sql("select * from myTempView")
    return data

create_df_from_rdd().show()
create_df_from_rdd().printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+----+---+-----+
| id|name|age| text|
+---+----+---+-----+
|  1|Miya| 10|water|
|  2|Nana| 30|drink|
|  3|Nana| 26|berry|
+---+----+---+-----+

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- text: string (nullable = true)

In [18]:
# csv創建表

def create_df_from_csv():
    df_from_csv = spark.read.csv('hdfs://10.120.26.200/user/spark/spark_sql_101/crime/data/', header=True, inferSchema=True)
    return df_from_csv

create_df_from_csv().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+--------------------+--------------------+-----+----+-----+
|lsoa_code|             borough|      major_category|      minor_category|value|year|month|
+---------+--------------------+--------------------+--------------------+-----+----+-----+
|E01004563|          Wandsworth|             Robbery|   Personal Property|    0|2008|    6|
|E01003496|              Newham|     Criminal Damage|Criminal Damage T...|    0|2013|    9|
|E01000086|Barking and Dagenham|  Theft and Handling|  Other Theft Person|    1|2009|    5|
|E01001317|              Ealing|            Burglary|Burglary in a Dwe...|    0|2013|    9|
|E01000713|             Bromley|Violence Against ...|    Offensive Weapon|    0|2009|   12|
+---------+--------------------+--------------------+--------------------+-----+----+-----+
only showing top 5 rows

In [19]:
# pandas創建表
import pandas as pd
import numpy as np

def create_df_from_pandas():
    df = pd.DataFrame(np.random.random((3, 4)))
    df_from_pandas = spark.createDataFrame(df, schema=['a', 'b', 'c', 'd'])
    return df_from_pandas
    
create_df_from_pandas().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------------+-------------------+------------------+
|                  a|                 b|                  c|                 d|
+-------------------+------------------+-------------------+------------------+
| 0.8205575047432271|0.5478281615029911| 0.8817539078301593|0.6536857088691732|
| 0.6370703218878147|0.6398977672193277|  0.549795106315854|0.8065077341224987|
|0.25685015746712214|0.5316804678523885|0.46278865500266375|0.4203465970220044|
+-------------------+------------------+-------------------+------------------+